# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(3)
# CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 200

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
#  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
 'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
 'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[39.20587486 50.08853996 35.37559313]
 [46.55235012 59.84495274 42.59752066]
 [72.245288   90.96648334 62.16188918]] [[  92.94859855  162.47584338  589.93066379]
 [ 199.32553707  339.36510156  926.26811452]
 [ 988.84055289 1587.19440903 2584.57344204]]
Mean and variance endomwnets:
[[ 7.47731355 31.62648643 34.84660757]
 [ 8.92689891 37.88511292 42.68660077]
 [13.61348708 57.09146433 63.15584145]] [[   3.49659948   66.09740516  580.48954987]
 [   7.61656482  130.85082927  963.74654124]
 [  34.18610677  659.01948972 2581.91028622]]
Mean and variance endomwnets:
[[ 4.25584282 52.99767184 34.39917429]
 [ 5.15490496 65.31671668 40.29901029]
 [ 7.57768503 97.97768305 62.76702875]] [[1.12200259e+00 1.74004003e+02 6.08319619e+02]
 [2.40380921e+00 3.78433348e+02 8.86329133e+02]
 [1.14630121e+01 1.86728929e+03 2.75482193e+03]]


# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = { 'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG),
                    'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG),
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG),
                    'Budget': lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

Guardrail-0.5

env reset!
starting_state [392.3      392.3      392.3      392.3      392.3       10.932322
   7.617694  10.668706]
Lower and Upper Solutions:
[[0.13917 0.14981 0.25356 0.15351 0.     ]
 [0.23113 0.21565 0.      0.21047 0.     ]
 [0.09738 0.10151 0.18536 0.10281 0.33458]]
[[0.3285  0.35372 0.60246 0.3625  0.     ]
 [0.54705 0.51015 0.      0.49778 0.     ]
 [0.23072 0.24067 0.43707 0.24381 0.79162]]
timestep:  0
new state [382.08      381.97922   381.04694   381.94394   383.84515     7.2902865
  13.422441    1.0207489]
reward:  -0.9003692079730182
timestep:  1
new state [372.10687  372.30737  376.20197  372.3709   383.02676   18.746086
  22.220646  32.167324]
reward:  -0.7707156249571203
timestep:  2
new state [263.3      263.3      263.3      263.3      263.3       34.28867
  41.664326  77.80902 ]
reward:  -0.8999260161235547

env reset!
starting_state [263.3       263.3       263.3       263.3       263.3         8.275917
   6.1462727  23.933739 ]
timestep:  0
new sta

timestep:  0
new state [777.6495  778.1482  791.07965 778.2958  827.68884  75.25718  84.63726
 166.01964]
reward:  -0.846738367822145
timestep:  1
new state [668.32263  668.3946   673.1357   668.40704  696.18445   28.833546
  56.928272 183.26923 ]
reward:  -0.9200261888742641
timestep:  2
new state [632.4      632.4      632.4      632.4      632.4       50.193836
  74.47444   70.67178 ]
reward:  -0.9548446090371185

env reset!
starting_state [632.4     632.4     632.4     632.4     632.4      68.8054   84.22174
 127.70535]
timestep:  0
new state [534.25977  534.36163  535.08923  534.3983   531.2294    67.63453
  54.094967 143.87057 ]
reward:  -0.902883538754635
timestep:  1
new state [449.25537  448.21606  431.43356  447.87634  417.27768   63.066257
  71.81964   87.96967 ]
reward:  -0.938002654433394
timestep:  2
new state [605.1      605.1      605.1      605.1      605.1        8.978854
   6.497436   7.013225]
reward:  -0.8893718790254584

env reset!
starting_state [605.1      605.1

timestep:  0
new state [260.7613   260.21527  297.5957   260.04694  286.53488   35.394726
  60.797546  69.90704 ]
reward:  -1.2115474344111565
timestep:  1
new state [199.74588  199.85506  245.68713  199.90851  231.14697   30.988016
  41.87631   13.387293]
reward:  -0.8822384412401337
timestep:  2
new state [477.8     477.8     477.8     477.8     477.8      29.97029  79.58301
  71.92901]
reward:  -0.8174208175723034

env reset!
starting_state [477.8      477.8      477.8      477.8      477.8       29.97012
  47.961544  19.105997]
timestep:  0
new state [437.3093    438.13312   451.36954   438.4033    462.63632    44.03313
  61.8908      7.9277773]
reward:  -0.8209004714661026
timestep:  1
new state [387.15796 389.07614 421.3454  389.7004  456.3076   71.95437  68.23424
 121.90351]
reward:  -0.7917693910691611
timestep:  2
new state [714.6      714.6      714.6      714.6      714.6       71.721085
  91.92644  256.4721  ]
reward:  -0.9139102138775407

env reset!
starting_state [714.6  

timestep:  1
new state [545.87335  545.7317   544.5032   545.6818   548.9283    46.625374
  40.536503  65.2993  ]
reward:  -0.8396585025478865
timestep:  2
new state [454.4     454.4     454.4     454.4     454.4      75.72425  94.26665
  65.86504]
reward:  -0.9076098209698429

env reset!
starting_state [454.4      454.4      454.4      454.4      454.4       16.239038
  26.541824  42.66182 ]
timestep:  0
new state [424.70282  424.84818  425.95715  424.89996  420.60666   18.141928
   6.920915  28.72264 ]
reward:  -0.9062699301163597
timestep:  1
new state [408.33023  407.98764  402.4701   407.87555  397.85672   20.395384
  17.43484   28.010838]
reward:  -0.9552226060752397
timestep:  2
new state [292.1     292.1     292.1     292.1     292.1      52.04399  50.57515
  69.5221 ]
reward:  -0.9074862298578679

env reset!
starting_state [292.1     292.1     292.1     292.1     292.1      44.41743  53.63957
  93.22526]
timestep:  0
new state [264.44244  264.41516  263.53046  264.40747  260.8

timestep:  2
new state [221.7       221.7       221.7       221.7       221.7         9.273838
   7.4962244   6.80624  ]
reward:  -0.8756082906627898

env reset!
starting_state [221.7      221.7      221.7      221.7      221.7       18.730669
  12.072046  12.40901 ]
timestep:  0
new state [215.09465  215.03098  214.64449  215.00807  217.53279   14.357786
  11.766319  30.252405]
reward:  -1.7465311532838894
timestep:  1
new state [196.96152  196.6689   192.76619  196.5705   207.39787    5.742756
   8.286935   8.829516]
reward:  -1.0382833444533834
timestep:  2
new state [170.       170.       170.       170.       170.        14.507949
  18.000618  64.7807  ]
reward:  -0.8784354063785779

env reset!
starting_state [170.        170.        170.        170.        170.          5.2659383
   8.845427    8.063645 ]
timestep:  0
new state [166.43745  166.48505  167.16566  166.5009   167.29501    5.69863
   7.238175  16.642036]
reward:  -1.727733961079378
timestep:  1
new state [162.35081  1

...


iter 91
episode 0
step 0
oldState [421.00302  421.47806  426.879    421.64157  420.18964   42.640495
  66.17611   12.99464 ]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.7968364281292858
newState [501.39282  502.63     523.1981   503.03342  544.2587    42.420963
  42.42093  149.11977 ]
info {'type': array([ 42.42096342,  42.42093047, 149.11976329])}

iter 91
episode 0
step 1
oldState [501.39282  502.63     523.1981   503.03342  544.2587    42.420963
  42.42093  149.11977 ]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.9563161477492784
newState [429.84625   430.09515   432.4442    430.18262   426.1701      3.754453
  10.728432    7.5455666]
info {'type': array([ 3.75445302, 10.72843143,  7.54556638

info {'type': array([11.3449239 , 14.51943186, 32.22775095])}

iter 121
episode 0
step 1
oldState [472.20398  471.76077  465.91     471.61142  467.60263   11.344924
  14.519432  32.227753]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.9285040263812026
newState [453.09872  452.5845   444.9821   452.41394  442.07758   58.592712
  95.66263  123.06715 ]
info {'type': array([ 58.59271254,  95.66262732, 123.06714403])}

iter 121
episode 0
step 2
oldState [453.09872  452.5845   444.9821   452.41394  442.07758   58.592712
  95.66263  123.06715 ]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.8904486076543321
newState [443.5      443.5      443.5      443.5      443.5       29.078756
  30.836649 125.66759 ]
i

newState [390.68286 390.66434 389.55457 390.6611  384.5261   44.67073  99.16755
  58.75831]
info {'type': array([44.67073204, 99.16754622, 58.75830875])}

iter 151
episode 0
step 2
oldState [390.68286 390.66434 389.55457 390.6611  384.5261   44.67073  99.16755
  58.75831]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.834591982534829
newState [364.3      364.3      364.3      364.3      364.3       11.45339
  22.747179  41.27174 ]
info {'type': array([11.45339017, 22.74717898, 41.27174019])}

iter 152
episode 0
step 0
oldState [138.36005  139.2631   144.91911  139.58904  170.7311    27.08266
  51.075848  47.723213]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.8669552895099502
newState [316.4516   31

iter 180
episode 0
step 2
oldState [228.74069  228.35397  219.62927  228.23589  240.6359    35.116917
  17.87208   52.06892 ]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.9399715713804938
newState [469.5      469.5      469.5      469.5      469.5       33.133133
  48.93146   74.913124]
info {'type': array([33.13313439, 48.93146102, 74.91312507])}

iter 181
episode 0
step 0
oldState [310.72058  311.82     329.34534  312.1814   343.79437   25.460798
  44.391293  15.7513  ]
action [[3.2850e-01 3.5372e-01 6.0246e-01 3.6250e-01 5.0000e-04]
 [5.4705e-01 5.1015e-01 5.0000e-04 4.9778e-01 5.0000e-04]
 [2.3072e-01 2.4067e-01 4.3707e-01 2.4381e-01 7.9162e-01]]
reward -0.8132498648373071
newState [433.21774  434.05692  447.25427  434.33304  456.99603   33.008034
  49.12839   50.76525 ]
info {'type': array([33.00803419, 49.12839024, 50.76525088])}

iter

ValueError: Parameter value must be nonnegative.

In [ ]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [ ]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)